<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/modelo_transcripcion_segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install openai-whisper pyannote.audio spacy
!python -m spacy download en_core_web_sm

# Importar librerías
from pyannote.audio import Pipeline
from whisper import load_model
import spacy
import os

# Configurar modelos y tokens
HUGGINGFACE_TOKEN = "tu_token_huggingface"
whisper_model = load_model("base")
nlp = spacy.load("en_core_web_sm")

# Descargar el pipeline de diarización
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token=HUGGINGFACE_TOKEN
)

# Función para procesar un archivo de audio
def process_audio(file_path):
    # Diarización
    diarization = diarization_pipeline(file_path)
    person_mapping = {}

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start, end = int(turn.start * 1000), int(turn.end * 1000)
        text = whisper_model.transcribe(file_path, task="transcribe", segment=(start, end))['text']

        # Analizar texto para detectar nombres
        doc = nlp(text)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        if names:
            person_mapping[speaker] = names[0]

    return {"person_mapping": person_mapping}

# Subir archivo de audio
from google.colab import files
uploaded = files.upload()

# Procesar archivo subido
for file_name in uploaded.keys():
    result = process_audio(file_name)
    print(result)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
